In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
df = pd.read_csv("../../data/Dataset3/data2.csv")

In [3]:
df.columns

Index(['id', 'country_id', 'league_id', 'season', 'date', 'home_team_goal',
       'away_team_goal', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH',
       'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD',
       'WHA', 'SJH', 'SJD', 'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA',
       'BSH', 'BSD', 'BSA'],
      dtype='object')

In [4]:
df

,id,country_id,league_id,season,date,home_team_goal,away_team_goal,B365H,B365D,B365A,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,2008-08-17 00:00:00,1,1,1.73,3.40,5.00,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,2008-08-16 00:00:00,0,0,1.95,3.20,3.60,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,2008-08-16 00:00:00,0,3,2.38,3.30,2.75,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,2008-08-17 00:00:00,5,0,1.44,3.75,7.50,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,2008-08-16 00:00:00,1,3,5.00,3.50,1.65,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,2015-09-22 00:00:00,1,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,2015-09-23 00:00:00,1,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,2015-09-23 00:00:00,2,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,2015-09-22 00:00:00,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
countries = pd.read_csv("../../data/Dataset3/country.csv")

In [6]:
countries

,country_id,country_name,country_code
0,1,Belgium,BEL
1,1729,England,GBR
2,4769,France,FRA
3,7809,Germany,DEU
4,10257,Italy,ITA
5,13274,Netherlands,NLD
6,15722,Poland,POL
7,17642,Portugal,PRT
8,19694,Scotland,SCOT
9,21518,Spain,ESP


In [7]:
df = df.merge(countries, on="country_id")

## One method

In [10]:
def calculate(df, colh, cold, cola):

    dfbw = df[["id", "country_name", "country_code", "league_id", "season","home_team_goal", "away_team_goal", colh, cold, cola]]

    def print_missing(row):
        if pd.isna(row[colh]):
            return row["country_name"]
    
    
    missing_countries = dfbw.apply(print_missing, axis=1)
    
    #print(missing_countries.dropna().value_counts())
    
    dfbw = dfbw.dropna(subset=[colh])
    
    def determine_final_result(row):
        hg = int(row["home_team_goal"])
        ag = int(row["away_team_goal"])
        if hg > ag:
            row["result"] = "1"
        elif hg == ag:
            row["result"] = "x"
        else:
            row["result"] = "2"
        return row

    def determine_bookie_prediction(row):
        h = row[colh]
        d = row[cold]
        a = row[cola]
        
        if h == min([h, d, a]):
            row["prediction"] = "1"
            row["prediction_performance"] = h
        elif d == min([h, d, a]):
            row["prediction"] = "x"
            row["prediction_performance"] = d
        else:
            row["prediction"] = "2"
            row["prediction_performance"] = a
        return row


    dfbw = dfbw.apply(determine_final_result, axis = 1)

    dfbw = dfbw.apply(determine_bookie_prediction, axis = 1)

    def determine_correct_prediction(row):
        row["correct"] = (row["result"] == row["prediction"])
        return row

    dfbw = dfbw.apply(determine_correct_prediction, axis = 1)

    groupedbw = dfbw.groupby(["country_name", "country_code", "season"])
        
    accuracies = []
    for name, group in groupedbw:
        total_acc = group["correct"].sum() / len(group["correct"]) * 100
        #print("Group: ", name, " accuracy: ", total_acc)
        accuracies.append((name[0], name[1], name[2], total_acc))

    test = pd.DataFrame(accuracies)

    test.columns = ["country_code", "country_name", "season", "accuracy_percentage"]
    
    by_season = test.groupby("season")
    
    folder = colh[:-1].lower()
    
    alls = set(countries["country_code"])
    
    for name, group in by_season:
        special = group
        actual = set(special["country_name"])
        for e in alls.difference(actual):
            special = special.append({"country_code" : e, "country_name" : e, "season" : special["season"].iloc[0], "accuracy_percentage" : 0.0}, ignore_index=True)
        special.to_csv("{}/{}.csv".format(folder, name.replace("/","_")), index=False)

    return

In [11]:
print(calculate(df, "BWH", "BWD", "BWA"))

None


In [12]:
print(calculate(df, "B365H", "B365D", "B365A"))

None


In [13]:
print(calculate(df, "IWH", "IWD", "IWA"))

None


In [14]:
print(calculate(df, "LBH", "LBD", "LBA"))

None


In [15]:
print(calculate(df, "PSH", "PSD", "PSA"))

None


In [16]:
print(calculate(df, "WHH", "WHD", "WHA"))

None


In [17]:
calculate(df, "SJH", "SJD", "SJA")

In [18]:
calculate(df, "VCH", "VCD", "VCA")

In [19]:
calculate(df, "GBH", "GBD", "GBA")

In [20]:
calculate(df, "BSH", "BSD", "BSA")